# Сохранение данных в Firebase

In [ ]:
# !pip install firebase_admin

In [1]:
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
import pandas as pd
import json

### Коннектимся с базой

In [2]:
cred = credentials.Certificate("CEC.json")
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': "https://parsingcec-default-rtdb.europe-west1.firebasedatabase.app/"
    })

In [3]:
ref = db.reference('/')

In [11]:
constRF= pd.read_csv("https://raw.githubusercontent.com/EmilStasevski/Parsing-CEC-Central-Election-Commission-of-the-Russian-Federation/main/data/republic_of_adygeya.csv")
president = pd.read_csv("https://raw.githubusercontent.com/EmilStasevski/Parsing-CEC-Central-Election-Commission-of-the-Russian-Federation/main/data/df.csv")


### Превращаем df в json

In [13]:
with open('constRF.json', 'w', encoding='utf-8') as file:
    constRF.to_json(file, force_ascii=False, orient='records')
    
with open('PresidentElection.json', 'w', encoding='utf-8') as file:
    president.to_json(file, force_ascii=False, orient='records')    

In [14]:
f1 = open('PresidentElection.json', encoding='utf-8')
f2 = open('constRF.json', encoding='utf-8')

data1 = json.load(f1)
data2 = json.load(f2)

### Пушим json в базу данных

In [15]:
#Смотрим, что внутри
data1[0]

{'candidate': 'БАБУРИН Сергей Николаевич',
 'voices': 479013,
 'percentage': 0.65,
 'year': 2018}

In [16]:
ref.set({
    # Президентские выборы
    "PresidentElection":
    data1,
    
    # поправки в Конституцию
    "ConstRF":
    data2
    
})

### Всем в одноя ячейке автоматически